In [1]:

import sqlite3
# import os.path
import hashlib
import pandas as pd
import numpy as np


db = './../data/daf06fb3ab69f27bd681a63311722181.db'

db = './test.db'


conn = sqlite3.connect(db)
df = pd.read_sql_query("SELECT * FROM loadsonly;", conn)
conn.close()# close db

df['datetime'] = pd.to_datetime(df['datetime'])
df.sort_values(by='datetime',ascending=True);


In [2]:
df['dp'] = df['demand'] - df['demand'].shift(1)
df['dp+'] = [0-x if x <=0 else 0 for x in df['dp']]
df['dp-'] = [x if x >=0 else 0 for x in df['dp']]

df['dp'] = df['demand'] - df['demand'].shift(1)
for x in range(1,len(df)):
    df.loc[x,'dp'] = df.loc[x,'demand'] -  df.loc[x-1,'demand']
    if df.loc[x,'dp'] > 0:
        df.loc[x,'dp+'] = df.loc[x,'dp']
        df.loc[x,'dp-'] = 0
    else:
        df.loc[x,'dp-'] = 0-df.loc[x,'dp']
        df.loc[x,'dp+'] = 0

In [3]:
# add user defined loads
user_loads = {'modem':10,'rPi':4}
baseload = 0
for user_load in user_loads:
    df.loc[:,user_load]  = user_loads[user_load]
    baseload = baseload + user_loads[user_load]
#     df.loc[:,] = np.array([df['demand'].min()] * len(df))


In [4]:
# find baseload and determine remaining unkwown profile
df.loc[:,'baseload'] = np.array([df['demand'].min()-baseload] * len(df))
df.loc[:,'unknown'] = df.loc[:,'demand'] - df.loc[:,'baseload']-baseload


In [5]:
# df[pd.isnull(df).any(axis=1)]

In [6]:
def left_round(number,precision=2):
    rem = number

    d = len(str(number))
    while(round(rem,d) == round(number,d)):
        rem = round(number,d-precision)
        d = d -1
        
    i = 0
    while(rem > 10**precision):
        rem = round(rem / 10,0)
        i = i +1
#         print(rem*(10**i))
        if i > 100:
            break
    return rem*(10**i)

left_round(00.00034564,3)

left_round(122431234.00034564,3)

# left_round(123456)

122000000.0

In [7]:
#find square loads
loads = dict()

for x in range(1,len(df)):
    #filter for dP+ with minimim power
    if df.loc[x,'dp+'] > 100:
        print(' ') #new line in output
        print ('start dp>100:',df.loc[x,'datetime'],'start P:',df.loc[x-1,'unknown'],'dP+:' ,df.loc[x,'dp+'])
        
        #look forwar in time to see how long load remains on
        for y in range(x+1,len(df)):
            #find dP- of same/simulair size 
            if df.loc[y,'dp-'] > 0.9 *df.loc[x,'dp+'] and df.loc[y,'dp-'] < 1.1 *df.loc[x,'dp+']:
                print ('stop dP-:',df.loc[y,'datetime'],'dP-:' ,df.loc[y,'dp-'])
                break
            # check if does not drop below start power (this means is must have been turned of)    
            if df.loc[y,'unknown'] <= df.loc[x-1,'unknown']:
                print ('stop demand:',df.loc[x,'datetime'] ,df.loc[y,'unknown'], ' <= ',df.loc[x-1,'unknown'])
                break
        
        print(x,y,(df.loc[x:y-1,'unknown']-df.loc[x-1,'unknown']).mean())
        
        # calculate key parameters
        mean = (df.loc[x:y-1,'unknown']-df.loc[x-1,'unknown']).mean()
        max = (df.loc[x:y-1,'unknown']-df.loc[x-1,'unknown']).max()
        min = (df.loc[x:y-1,'unknown']-df.loc[x-1,'unknown']).min()
        std = (df.loc[x:y-1,'unknown']-df.loc[x-1,'unknown']).std()
        time = pd.Timedelta(pd.to_datetime(df['datetime'][y])-pd.to_datetime(df['datetime'][x]))
        noise = df.loc[x:y-1,'dp'].abs().mean()
        
        print('mean load:',mean)
        print('max load:',max)
        print('std load:',std)
        print('noise:',noise)
        print('time past:',time)


        print('found profile from ',df.loc[x-1,'datetime'],'to',df.loc[y,'datetime'])
        profile = list(df.loc[x-1:y,'unknown']-df.loc[x-1,'unknown'])
        print(profile)
        
        
#         plt.plot(range(x,y),profile)
#         plt.title('A simple chirp')
#         plt.show()
#         break
        try:
            #drop loads which do not have a square looking profile.
            if (std/mean > 0.1):
                print('dropped because of large std:',std/mean)
                continue

            #drop loads which do not have odd start compared to mean
            if( abs(mean-df.loc[x,'dp+'])/df.loc[x,'dp+'] > 0.2):    
                print('dropped because large difference between mean and start dp+:',abs(mean-df.loc[x,'dp+'])/df.loc[x,'dp+'])
                continue
        except:
            print('Something went wrong in analysing found load, dropping ....')
            continue
#         print('hist:',df.hist(bins=10))
        
        # save this load to the distionairy (group by 'fingerprint')
        fingerprint = str(int(round(mean,-2)))+'_'#+str(int(left_round(df.loc[x,'dp+'],2)))
        temp = {'index':{'start':x,'stop':y},'time':time,'mean':mean,'max':max,'profile':profile}
        try:
            loads[fingerprint][len(loads[fingerprint])] = temp
        except:
            loads[fingerprint] = {0:temp}
            df[fingerprint] = 0 #add new colum to the dataframe before inserting found loads (only on first occurance!)
        
        df.loc[x:y-1,fingerprint] = profile[1:-1]
        df.loc[x:y-1,'unknown'] = df.loc[x:y-1,'unknown'] - profile[1:-1]
        print('Saved with fingerprint:',fingerprint)



 
start dp>100: 2017-09-21 19:32:09 start P: 173.0 dP+: 133.0
stop dP-: 2017-09-21 20:15:10 dP-: 120.0
149 409 230.33076923076922
mean load: 230.33076923076922
max load: 1071.0
std load: 285.376277743999
noise: 8.696153846153846
time past: 0 days 00:43:01
found profile from  2017-09-21 19:31:49 to 2017-09-21 20:15:10
[0.0, 133.0, 133.0, 179.0, 179.0, 180.0, 180.0, 181.0, 181.0, 179.0, 179.0, 179.0, 179.0, 169.0, 169.0, 170.0, 170.0, 168.0, 168.0, 168.0, 168.0, 167.0, 167.0, 167.0, 167.0, 168.0, 168.0, 167.0, 167.0, 172.0, 172.0, 173.0, 173.0, 173.0, 173.0, 160.0, 160.0, 160.0, 160.0, 161.0, 161.0, 155.0, 155.0, 154.0, 154.0, 152.0, 152.0, 149.0, 149.0, 161.0, 161.0, 969.0, 969.0, 970.0, 970.0, 1071.0, 1071.0, 1069.0, 1069.0, 1068.0, 1068.0, 1002.0, 1002.0, 1001.0, 1001.0, 996.0, 996.0, 990.0, 990.0, 989.0, 989.0, 990.0, 990.0, 992.0, 992.0, 988.0, 988.0, 983.0, 983.0, 982.0, 982.0, 174.0, 174.0, 174.0, 174.0, 174.0, 174.0, 173.0, 173.0, 175.0, 175.0, 175.0, 175.0, 175.0, 175.0, 175.0, 

3227 3411 304.0
mean load: 304.0
max load: 2032.0
std load: 410.02878747397364
noise: 51.60326086956522
time past: 0 days 00:30:27
found profile from  2017-09-22 04:01:05 to 2017-09-22 04:31:51
[0.0, 108.0, 108.0, 107.0, 107.0, 110.0, 110.0, 109.0, 109.0, 105.0, 105.0, 102.0, 102.0, 98.0, 98.0, 95.0, 95.0, 142.0, 142.0, 142.0, 142.0, 142.0, 142.0, 213.0, 213.0, 224.0, 224.0, 1059.0, 1059.0, 1059.0, 1059.0, 1077.0, 1077.0, 1075.0, 1075.0, 1084.0, 1084.0, 1080.0, 1080.0, 1089.0, 1089.0, 1092.0, 1092.0, 229.0, 229.0, 230.0, 230.0, 224.0, 224.0, 223.0, 223.0, 230.0, 230.0, 2032.0, 2032.0, 2024.0, 2024.0, 230.0, 230.0, 230.0, 230.0, 2014.0, 2014.0, 228.0, 228.0, 229.0, 229.0, 228.0, 228.0, 229.0, 229.0, 229.0, 229.0, 229.0, 229.0, 229.0, 229.0, 229.0, 229.0, 228.0, 228.0, 181.0, 181.0, 176.0, 176.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 169.0, 168.0, 168.0

stop demand: 2017-09-22 13:28:54 2.0  <=  2.0
6058 6130 88.83333333333333
mean load: 88.83333333333333
max load: 109.0
std load: 15.702336777743948
noise: 3.513888888888889
time past: 0 days 00:23:49
found profile from  2017-09-22 13:28:34 to 2017-09-22 13:52:43
[0.0, 107.0, 109.0, 107.0, 109.0, 106.0, 102.0, 99.0, 98.0, 94.0, 93.0, 91.0, 92.0, 90.0, 90.0, 89.0, 89.0, 89.0, 90.0, 89.0, 89.0, 88.0, 89.0, 90.0, 90.0, 90.0, 89.0, 89.0, 89.0, 89.0, 90.0, 90.0, 91.0, 90.0, 91.0, 91.0, 90.0, 91.0, 90.0, 89.0, 90.0, 90.0, 89.0, 90.0, 90.0, 90.0, 91.0, 91.0, 90.0, 90.0, 90.0, 89.0, 90.0, 89.0, 89.0, 88.0, 89.0, 89.0, 89.0, 90.0, 89.0, 88.0, 89.0, 88.0, 89.0, 89.0, 88.0, 88.0, 88.0, 88.0, 88.0, 2.0, 1.0, 0.0]
dropped because of large std: 0.17676176485265233
 
start dp>100: 2017-09-22 14:40:02 start P: 4.0 dP+: 108.0
stop demand: 2017-09-22 14:40:02 3.0  <=  4.0
6273 6343 90.01428571428572
mean load: 90.01428571428572
max load: 108.0
std load: 5.4573255039730455
noise: 2.2
time past: 0 days 00:

mean load: 2047.0652173913043
max load: 2120.0
std load: 12.637514126006907
noise: 50.67391304347826
time past: 0 days 00:15:13
found profile from  2017-09-22 19:51:23 to 2017-09-22 20:06:56
[0.0, 2048.0, 2049.0, 2046.0, 2048.0, 2055.0, 2055.0, 2050.0, 2044.0, 2043.0, 2042.0, 2046.0, 2046.0, 2049.0, 2055.0, 2052.0, 2040.0, 2041.0, 2120.0, 2055.0, 2053.0, 2049.0, 2053.0, 2052.0, 2048.0, 2047.0, 2047.0, 2052.0, 2049.0, 2048.0, 2047.0, 2041.0, 2049.0, 2044.0, 2042.0, 2050.0, 2044.0, 2042.0, 2042.0, 2036.0, 2040.0, 2037.0, 2033.0, 2033.0, 2035.0, 2035.0, 2033.0, -21.0]
Saved with fingerprint: 2000_
 
start dp>100: 2017-09-22 20:22:49 start P: 10.0 dP+: 1529.0
stop dP-: 2017-09-22 20:23:09 dP-: 1402.0
7309 7310 1529.0
mean load: 1529.0
max load: 1529.0
std load: nan
noise: 1529.0
time past: 0 days 00:00:20
found profile from  2017-09-22 20:22:29 to 2017-09-22 20:23:09
[0.0, 1529.0, 127.0]
Saved with fingerprint: 1500_
 
start dp>100: 2017-09-22 21:30:59 start P: 32.0 dP+: 107.0
stop demand:

stop dP-: 2017-09-23 07:44:28 dP-: 1739.0
9368 9369 1739.0
mean load: 1739.0
max load: 1739.0
std load: nan
noise: 1739.0
time past: 0 days 00:00:20
found profile from  2017-09-23 07:43:48 to 2017-09-23 07:44:28
[0.0, 1739.0, 0.0]
Saved with fingerprint: 1700_
 
start dp>100: 2017-09-23 07:47:46 start P: 13.0 dP+: 1781.0
stop dP-: 2017-09-23 07:48:06 dP-: 1780.0
9379 9380 1781.0
mean load: 1781.0
max load: 1781.0
std load: nan
noise: 1781.0
time past: 0 days 00:00:20
found profile from  2017-09-23 07:47:26 to 2017-09-23 07:48:06
[0.0, 1781.0, 1.0]
Saved with fingerprint: 1800_
 
start dp>100: 2017-09-23 07:48:26 start P: 14.0 dP+: 1779.0
stop dP-: 2017-09-23 07:48:46 dP-: 1779.0
9381 9382 1779.0
mean load: 1779.0
max load: 1779.0
std load: nan
noise: 1779.0
time past: 0 days 00:00:20
found profile from  2017-09-23 07:48:06 to 2017-09-23 07:48:46
[0.0, 1779.0, 0.0]
Saved with fingerprint: 1800_
 
start dp>100: 2017-09-23 07:49:25 start P: 14.0 dP+: 1687.0
stop dP-: 2017-09-23 07:49:45 d

...

found profile from  2017-09-26 18:23:56 to 2017-09-26 18:24:16
[0.0, 114.0, 2.0]
Saved with fingerprint: 100_
 
start dp>100: 2017-09-26 18:26:25 start P: 119.0 dP+: 135.0
stop dP-: 2017-09-26 18:26:35 dP-: 135.0
30414 30415 135.0
mean load: 135.0
max load: 135.0
std load: nan
noise: 135.0
time past: 0 days 00:00:10
found profile from  2017-09-26 18:26:15 to 2017-09-26 18:26:35
[0.0, 135.0, 0.0]
Saved with fingerprint: 100_
 
start dp>100: 2017-09-26 18:29:33 start P: 117.0 dP+: 181.0
stop demand: 2017-09-26 18:29:33 90.0  <=  117.0
30433 30463 97.76666666666667
mean load: 97.76666666666667
max load: 652.0
std load: 110.15559006690721
noise: 66.1
time past: 0 days 00:04:58
found profile from  2017-09-26 18:29:24 to 2017-09-26 18:34:31
[0.0, 181.0, 72.0, 62.0, 61.0, 60.0, 60.0, 62.0, 60.0, 60.0, 60.0, 158.0, 62.0, 62.0, 59.0, 141.0, 96.0, 62.0, 61.0, 60.0, 60.0, 61.0, 141.0, 652.0, 58.0, 60.0, 61.0, 61.0, 125.0, 94.0, 61.0, -27.0]
dropped because of large std: 1.126719298331816
 
start 

mean load: 97.40816326530613
max load: 300.0
std load: 61.12620767973522
noise: 32.16326530612245
time past: 0 days 00:08:06
found profile from  2017-09-26 19:01:30 to 2017-09-26 19:09:46
[0.0, 108.0, 147.0, 70.0, 72.0, 63.0, 63.0, 63.0, 64.0, 64.0, 63.0, 63.0, 136.0, 111.0, 70.0, 64.0, 62.0, 63.0, 126.0, 108.0, 64.0, 63.0, 62.0, 104.0, 63.0, 63.0, 63.0, 120.0, 97.0, 182.0, 225.0, 225.0, 217.0, 300.0, 274.0, 62.0, 62.0, 63.0, 95.0, 62.0, 63.0, 64.0, 104.0, 42.0, 15.0, 81.0, 82.0, 16.0, 130.0, 130.0, 0.0]
dropped because of large std: 0.6275265401858423
 
start dp>100: 2017-09-26 19:09:26 start P: 46.0 dP+: 114.0
stop demand: 2017-09-26 19:09:26 30.0  <=  46.0
30674 30676 114.0
mean load: 114.0
max load: 114.0
std load: 0.0
noise: 57.0
time past: 0 days 00:00:20
found profile from  2017-09-26 19:09:16 to 2017-09-26 19:09:46
[0.0, 114.0, 114.0, -16.0]
Saved with fingerprint: 100_
 
start dp>100: 2017-09-26 19:09:56 start P: 30.0 dP+: 159.0
stop demand: 2017-09-26 19:09:56 29.0  <=  30.0


31995 32137 90.99295774647888
mean load: 90.99295774647888
max load: 110.0
std load: 9.163987857870328
noise: 2.119718309859155
time past: 0 days 00:23:30
found profile from  2017-09-26 22:47:48 to 2017-09-26 23:11:28
[0.0, 109.0, 109.0, 109.0, 110.0, 108.0, 109.0, 109.0, 108.0, 106.0, 106.0, 101.0, 100.0, 99.0, 99.0, 96.0, 95.0, 94.0, 93.0, 94.0, 94.0, 93.0, 93.0, 93.0, 91.0, 90.0, 90.0, 90.0, 89.0, 89.0, 89.0, 91.0, 89.0, 90.0, 91.0, 90.0, 91.0, 90.0, 89.0, 91.0, 90.0, 89.0, 89.0, 89.0, 90.0, 90.0, 90.0, 90.0, 90.0, 91.0, 89.0, 89.0, 89.0, 90.0, 90.0, 90.0, 91.0, 91.0, 90.0, 92.0, 91.0, 92.0, 90.0, 92.0, 91.0, 91.0, 91.0, 91.0, 91.0, 91.0, 91.0, 92.0, 91.0, 91.0, 91.0, 91.0, 90.0, 92.0, 90.0, 90.0, 90.0, 91.0, 91.0, 90.0, 90.0, 90.0, 91.0, 90.0, 92.0, 91.0, 91.0, 91.0, 89.0, 90.0, 91.0, 89.0, 90.0, 90.0, 89.0, 90.0, 90.0, 91.0, 89.0, 90.0, 90.0, 89.0, 89.0, 90.0, 90.0, 90.0, 89.0, 89.0, 89.0, 89.0, 90.0, 90.0, 89.0, 89.0, 88.0, 89.0, 88.0, 89.0, 89.0, 89.0, 88.0, 88.0, 89.0, 88.0, 88

start dp>100: 2017-09-27 14:53:22 start P: 3.0 dP+: 111.0
stop demand: 2017-09-27 14:53:22 2.0  <=  3.0
37830 37972 90.49295774647888
mean load: 90.49295774647888
max load: 111.0
std load: 7.754885423220644
noise: 1.7605633802816902
time past: 0 days 00:23:30
found profile from  2017-09-27 14:53:13 to 2017-09-27 15:16:52
[0.0, 111.0, 110.0, 110.0, 110.0, 109.0, 110.0, 110.0, 108.0, 107.0, 105.0, 103.0, 101.0, 100.0, 99.0, 98.0, 95.0, 95.0, 94.0, 94.0, 93.0, 92.0, 92.0, 92.0, 92.0, 91.0, 90.0, 90.0, 89.0, 90.0, 90.0, 89.0, 90.0, 90.0, 90.0, 89.0, 90.0, 89.0, 89.0, 89.0, 89.0, 89.0, 89.0, 89.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 89.0, 88.0, 89.0, 89.0, 90.0, 90.0, 91.0, 90.0, 90.0, 90.0, 90.0, 89.0, 88.0, 91.0, 90.0, 92.0, 89.0, 90.0, 90.0, 91.0, 89.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 89.0, 89.0, 90.0, 91.0, 90.0, 90.0, 90.0, 89.0, 90.0, 89.0, 89.0, 89.0, 89.0, 90.0, 90.0, 89.0, 89.0, 89.0, 89.0, 89.0, 89.0, 89.0, 88.0, 88.0, 88.0, 89.0, 88.0, 88.0, 88.0, 87.0, 88.0

start dp>100: 2017-09-27 20:55:03 start P: 127.0 dP+: 108.0
stop demand: 2017-09-27 20:55:03 107.0  <=  127.0
40016 40207 79.50785340314137
mean load: 79.50785340314137
max load: 115.0
std load: 38.86506604975836
noise: 2.3769633507853403
time past: 0 days 00:31:36
found profile from  2017-09-27 20:54:53 to 2017-09-27 21:26:39
[0.0, 108.0, 111.0, 111.0, 112.0, 110.0, 112.0, 112.0, 111.0, 111.0, 115.0, 113.0, 113.0, 109.0, 106.0, 104.0, 104.0, 105.0, 103.0, 102.0, 102.0, 101.0, 101.0, 104.0, 100.0, 99.0, 98.0, 99.0, 102.0, 101.0, 104.0, 101.0, 101.0, 101.0, 100.0, 102.0, 103.0, 104.0, 101.0, 98.0, 98.0, 97.0, 101.0, 101.0, 100.0, 99.0, 99.0, 98.0, 99.0, 99.0, 100.0, 99.0, 95.0, 104.0, 99.0, 102.0, 100.0, 100.0, 101.0, 101.0, 103.0, 102.0, 104.0, 100.0, 101.0, 101.0, 100.0, 102.0, 101.0, 105.0, 103.0, 102.0, 100.0, 100.0, 104.0, 102.0, 101.0, 102.0, 101.0, 101.0, 101.0, 102.0, 102.0, 101.0, 101.0, 101.0, 102.0, 103.0, 100.0, 102.0, 100.0, 101.0, 103.0, 103.0, 104.0, 102.0, 101.0, 100.0, 

In [8]:
import plotly.offline
import plotly.graph_objs as go
import cufflinks
cufflinks.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [9]:
start = 0
points = 1000
try:
    start = plots * points
except:
    plots = 0
# plots = 0

    
res = df.set_index('datetime')
res[start:start+points].iplot(kind='line')

plots = plots +1
# df[].iplot()

In [ ]:
# print(list(loads['100_'][0]))
for i in sorted(loads):
    print(i,'has',len(loads[i]),'occurance(s)')
#     for load in loads[i]:
#         if i == '100_':
#             print(round(sum(loads[i][load]['profile'])/360,1),'Wh in ',loads[i][load]['time'])
            


100_ has 85 occurance(s)
1300_ has 2 occurance(s)
1400_ has 1 occurance(s)
1500_ has 2 occurance(s)
1600_ has 7 occurance(s)
1700_ has 10 occurance(s)
1800_ has 6 occurance(s)
1900_ has 1 occurance(s)
2000_ has 7 occurance(s)
200_ has 8 occurance(s)
2100_ has 2 occurance(s)
2600_ has 1 occurance(s)
2700_ has 2 occurance(s)
2800_ has 1 occurance(s)
300_ has 7 occurance(s)
400_ has 13 occurance(s)
500_ has 2 occurance(s)
800_ has 11 occurance(s)
900_ has 3 occurance(s)


In [ ]:

res = df.set_index('datetime')
# res[:1000].iplot(kind='line')
# fig = res.plot()
# fig.show()
res.resample('10T').iplot(kind='line')



# res[:100]

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning:


.resample() is now a deferred operation
You called iplot(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead



In [ ]:

# df[['datetime',list(loads)]].iplot()
temp = list(loads)
temp[0] = 'datetime'
temp[1] = 'baseload'
temp[2] = 'unknown'

temp[3:] =list(loads)
dp = df[temp]
dp = dp.set_index('datetime')
dp.resample('10T').mean().iplot(kind='line')
# dp.resample('10T').mean().iplot(kind='area',fill='tozeroy',mode='none')



In [ ]:
pie = dp.sum()
# values = pie.values/pie.sum()*100
round(pie/pie.sum()*100,1)


In [ ]:
trace = go.Pie(labels=pie.index, values=pie.values)
dp.iplot([trace], filename='disaggregator_results_pie')

In [ ]:
# import plotly.plotly as py
import plotly.offline as py

trace = go.Pie(labels=pie.index, values=pie.values)

py.plot([trace], filename='disaggregator_results_pie.html')

# Try to rename colums for 'kwown loads' 

In [ ]:
#relable output for known_loads
known_loads = {'100_':'Fridge','2000_':'Iron'}
for name in loads.keys():
    try:
        df=df.rename(columns = {name:known_loads[name]})
        loads[known_loads[name]] = loads[name]
    except:
        print(name,'not found known_loads')
    
for name in known_loads:
    try:
        del loads[name]
    except:
        print(name,'not found in legend')

legend = loads.keys()
print(list(legend))